# Traditional Knowledge Graph Completion Methods 
#### Author: Ridha Alkhbaz 
#### Imports:


In [1]:
import argparse
import os.path as osp

import torch
import torch.optim as optim

from torch_geometric.datasets import FB15k_237
from torch_geometric.nn import ComplEx, DistMult, RotatE, TransE


### DistMult Replication & Data Download:

In [4]:
# choose the model here 
model_type = 'distmult'
model_map = {
    'transe': TransE,
    'complex': ComplEx,
    'distmult': DistMult,
    'rotate': RotatE,
}

device = 'cuda' if torch.cuda.is_available() else 'cpu'
path = 'data'

train_data = FB15k_237(path, split='train')[0].to(device)
val_data = FB15k_237(path, split='val')[0].to(device)
test_data = FB15k_237(path, split='test')[0].to(device)

model_arg_map = {'rotate': {'margin': 9.0}}
model = model_map[model_type](
    num_nodes=train_data.num_nodes,
    num_relations=train_data.num_edge_types,
    hidden_channels=50,
    **model_arg_map.get(model_type, {}),
).to(device)

loader = model.loader(
    head_index=train_data.edge_index[0],
    rel_type=train_data.edge_type,
    tail_index=train_data.edge_index[1],
    batch_size=1000,
    shuffle=True,
)

optimizer_map = {
    'transe': optim.Adam(model.parameters(), lr=0.01),
    'complex': optim.Adagrad(model.parameters(), lr=0.001, weight_decay=1e-6),
    'distmult': optim.Adam(model.parameters(), lr=0.0001, weight_decay=1e-6),
    'rotate': optim.Adam(model.parameters(), lr=1e-3),
}
optimizer = optimizer_map[model_type]


def train():
    model.train()
    total_loss = total_examples = 0
    for head_index, rel_type, tail_index in loader:
        optimizer.zero_grad()
        loss = model.loss(head_index, rel_type, tail_index)
        loss.backward()
        optimizer.step()
        total_loss += float(loss) * head_index.numel()
        total_examples += head_index.numel()
    return total_loss / total_examples


@torch.no_grad()
def test(data):
    model.eval()
    return model.test(
        head_index=data.edge_index[0],
        rel_type=data.edge_type,
        tail_index=data.edge_index[1],
        batch_size=20000,
        k=10,
    )

# training and testing Distmult 
for epoch in range(1, 201):
    loss = train()
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}')
    if epoch % 25 == 0:
        rank, mrr = test(val_data)
        print(f'Epoch: {epoch:03d}, Val Mean Rank: {rank:.2f}, '
              f'Val MRR: {mrr:.4f}')
    if epoch % 100 == 0:
        rank, mrr = test(test_data)
        print(f'Test Mean Rank: {rank:.2f}, Test MRR: {mrr:.4f}, ')
        mrr = round(mrr*100, 4)
        torch.save(model.state_dict(), './mods/distmult_mods/distmult_fbk_epch'+str(epoch+100)+'_mrr'+str(mrr)+'.pt')


Epoch: 001, Loss: 1.0000
Epoch: 002, Loss: 1.0000
Epoch: 003, Loss: 0.9999
Epoch: 004, Loss: 0.9997
Epoch: 005, Loss: 0.9986
Epoch: 006, Loss: 0.9949
Epoch: 007, Loss: 0.9859
Epoch: 008, Loss: 0.9691
Epoch: 009, Loss: 0.9426
Epoch: 010, Loss: 0.9050
Epoch: 011, Loss: 0.8562
Epoch: 012, Loss: 0.8016
Epoch: 013, Loss: 0.7470
Epoch: 014, Loss: 0.6936
Epoch: 015, Loss: 0.6423
Epoch: 016, Loss: 0.5942
Epoch: 017, Loss: 0.5510
Epoch: 018, Loss: 0.5109
Epoch: 019, Loss: 0.4748
Epoch: 020, Loss: 0.4433
Epoch: 021, Loss: 0.4152
Epoch: 022, Loss: 0.3890
Epoch: 023, Loss: 0.3666
Epoch: 024, Loss: 0.3461
Epoch: 025, Loss: 0.3278


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 17535/17535 [00:42<00:00, 413.96it/s]


Epoch: 025, Val Mean Rank: 590.53, Val MRR: 0.3496
Epoch: 026, Loss: 0.3105
Epoch: 027, Loss: 0.2939
Epoch: 028, Loss: 0.2803
Epoch: 029, Loss: 0.2677
Epoch: 030, Loss: 0.2555
Epoch: 031, Loss: 0.2441
Epoch: 032, Loss: 0.2330
Epoch: 033, Loss: 0.2240
Epoch: 034, Loss: 0.2145
Epoch: 035, Loss: 0.2066
Epoch: 036, Loss: 0.1999
Epoch: 037, Loss: 0.1914
Epoch: 038, Loss: 0.1846
Epoch: 039, Loss: 0.1792
Epoch: 040, Loss: 0.1729
Epoch: 041, Loss: 0.1670
Epoch: 042, Loss: 0.1622
Epoch: 043, Loss: 0.1577
Epoch: 044, Loss: 0.1514
Epoch: 045, Loss: 0.1492
Epoch: 046, Loss: 0.1436
Epoch: 047, Loss: 0.1410
Epoch: 048, Loss: 0.1359
Epoch: 049, Loss: 0.1328
Epoch: 050, Loss: 0.1301


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 17535/17535 [00:43<00:00, 407.38it/s]


Epoch: 050, Val Mean Rank: 461.15, Val MRR: 0.3690
Epoch: 051, Loss: 0.1266
Epoch: 052, Loss: 0.1238
Epoch: 053, Loss: 0.1212
Epoch: 054, Loss: 0.1188
Epoch: 055, Loss: 0.1151
Epoch: 056, Loss: 0.1131
Epoch: 057, Loss: 0.1109
Epoch: 058, Loss: 0.1089
Epoch: 059, Loss: 0.1060
Epoch: 060, Loss: 0.1058
Epoch: 061, Loss: 0.1033
Epoch: 062, Loss: 0.1009
Epoch: 063, Loss: 0.1003
Epoch: 064, Loss: 0.0977
Epoch: 065, Loss: 0.0958
Epoch: 066, Loss: 0.0948
Epoch: 067, Loss: 0.0927
Epoch: 068, Loss: 0.0923
Epoch: 069, Loss: 0.0906
Epoch: 070, Loss: 0.0888
Epoch: 071, Loss: 0.0875
Epoch: 072, Loss: 0.0864
Epoch: 073, Loss: 0.0862
Epoch: 074, Loss: 0.0846
Epoch: 075, Loss: 0.0841


...

Epoch: 175, Val Mean Rank: 284.53, Val MRR: 0.3956
Epoch: 176, Loss: 0.0464
Epoch: 177, Loss: 0.0448
Epoch: 178, Loss: 0.0455
Epoch: 179, Loss: 0.0459
Epoch: 180, Loss: 0.0461
Epoch: 181, Loss: 0.0456
Epoch: 182, Loss: 0.0447
Epoch: 183, Loss: 0.0457
Epoch: 184, Loss: 0.0443
Epoch: 185, Loss: 0.0447
Epoch: 186, Loss: 0.0441
Epoch: 187, Loss: 0.0441
Epoch: 188, Loss: 0.0445
Epoch: 189, Loss: 0.0443
Epoch: 190, Loss: 0.0437
Epoch: 191, Loss: 0.0438
Epoch: 192, Loss: 0.0441
Epoch: 193, Loss: 0.0432
Epoch: 194, Loss: 0.0434
Epoch: 195, Loss: 0.0439
Epoch: 196, Loss: 0.0431
Epoch: 197, Loss: 0.0427
Epoch: 198, Loss: 0.0428
Epoch: 199, Loss: 0.0430
Epoch: 200, Loss: 0.0428


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 17535/17535 [00:42<00:00, 414.73it/s]


Epoch: 200, Val Mean Rank: 273.99, Val MRR: 0.3981


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20466/20466 [00:48<00:00, 422.17it/s]

Test Mean Rank: 287.34, Test MRR: 0.3936, 


#### TranSe Model 

In [5]:
# training and testing TransE
model_type = 'transe'
model_map = {
    'transe': TransE,
    'complex': ComplEx,
    'distmult': DistMult,
    'rotate': RotatE,
}

device = 'cuda' if torch.cuda.is_available() else 'cpu'
path = 'data'

train_data = FB15k_237(path, split='train')[0].to(device)
val_data = FB15k_237(path, split='val')[0].to(device)
test_data = FB15k_237(path, split='test')[0].to(device)

model_arg_map = {'rotate': {'margin': 9.0}}
model = model_map[model_type](
    num_nodes=train_data.num_nodes,
    num_relations=train_data.num_edge_types,
    hidden_channels=50,
    **model_arg_map.get(model_type, {}),
).to(device)

loader = model.loader(
    head_index=train_data.edge_index[0],
    rel_type=train_data.edge_type,
    tail_index=train_data.edge_index[1],
    batch_size=1000,
    shuffle=True,
)

optimizer_map = {
    'transe': optim.Adam(model.parameters(), lr=0.01),
    'complex': optim.Adagrad(model.parameters(), lr=0.001, weight_decay=1e-6),
    'distmult': optim.Adam(model.parameters(), lr=0.0001, weight_decay=1e-6),
    'rotate': optim.Adam(model.parameters(), lr=1e-3),
}
optimizer = optimizer_map[model_type]


def train():
    model.train()
    total_loss = total_examples = 0
    for head_index, rel_type, tail_index in loader:
        optimizer.zero_grad()
        loss = model.loss(head_index, rel_type, tail_index)
        loss.backward()
        optimizer.step()
        total_loss += float(loss) * head_index.numel()
        total_examples += head_index.numel()
    return total_loss / total_examples


@torch.no_grad()
def test(data):
    model.eval()
    return model.test(
        head_index=data.edge_index[0],
        rel_type=data.edge_type,
        tail_index=data.edge_index[1],
        batch_size=20000,
        k=10,
    )
# training and testing Distmult 
for epoch in range(1, 201):
    loss = train()
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}')
    if epoch % 25 == 0:
        rank, mrr = test(val_data)
        print(f'Epoch: {epoch:03d}, Val Mean Rank: {rank:.2f}, '
              f'Val MRR: {mrr:.4f}')
    if epoch % 100 == 0:
        rank, mrr = test(test_data)
        print(f'Test Mean Rank: {rank:.2f}, Test MRR: {mrr:.4f}, ')
        mrr = round(mrr*100, 4)
        torch.save(model.state_dict(), './mods/transe_mods/transe_fbk_epch'+str(epoch)+'_mrr'+str(mrr)+'.pt')

Epoch: 001, Loss: 0.7595
Epoch: 002, Loss: 0.5594
Epoch: 003, Loss: 0.4406
Epoch: 004, Loss: 0.3551
Epoch: 005, Loss: 0.3019
Epoch: 006, Loss: 0.2672
Epoch: 007, Loss: 0.2441
Epoch: 008, Loss: 0.2283
Epoch: 009, Loss: 0.2163
Epoch: 010, Loss: 0.2054
Epoch: 011, Loss: 0.2003
Epoch: 012, Loss: 0.1937
Epoch: 013, Loss: 0.1887
Epoch: 014, Loss: 0.1843
Epoch: 015, Loss: 0.1796
Epoch: 016, Loss: 0.1763
Epoch: 017, Loss: 0.1725
Epoch: 018, Loss: 0.1698
Epoch: 019, Loss: 0.1665
Epoch: 020, Loss: 0.1643
Epoch: 021, Loss: 0.1622
Epoch: 022, Loss: 0.1598
Epoch: 023, Loss: 0.1571
Epoch: 024, Loss: 0.1544
Epoch: 025, Loss: 0.1524


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 17535/17535 [04:19<00:00, 67.58it/s]


Epoch: 025, Val Mean Rank: 427.71, Val MRR: 0.3598
Epoch: 026, Loss: 0.1518
Epoch: 027, Loss: 0.1497
Epoch: 028, Loss: 0.1487
Epoch: 029, Loss: 0.1451
Epoch: 030, Loss: 0.1447
Epoch: 031, Loss: 0.1438
Epoch: 032, Loss: 0.1417
Epoch: 033, Loss: 0.1400
Epoch: 034, Loss: 0.1389
Epoch: 035, Loss: 0.1379
Epoch: 036, Loss: 0.1357
Epoch: 037, Loss: 0.1355
Epoch: 038, Loss: 0.1344
Epoch: 039, Loss: 0.1327
Epoch: 040, Loss: 0.1313
Epoch: 041, Loss: 0.1305
Epoch: 042, Loss: 0.1288
Epoch: 043, Loss: 0.1289
Epoch: 044, Loss: 0.1276
Epoch: 045, Loss: 0.1269
Epoch: 046, Loss: 0.1252
Epoch: 047, Loss: 0.1232
Epoch: 048, Loss: 0.1223
Epoch: 049, Loss: 0.1215
Epoch: 050, Loss: 0.1207


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 17535/17535 [04:28<00:00, 65.26it/s]


Epoch: 050, Val Mean Rank: 355.37, Val MRR: 0.3588
Epoch: 051, Loss: 0.1210
Epoch: 052, Loss: 0.1204
Epoch: 053, Loss: 0.1191
Epoch: 054, Loss: 0.1188
Epoch: 055, Loss: 0.1180
Epoch: 056, Loss: 0.1171
Epoch: 057, Loss: 0.1153
Epoch: 058, Loss: 0.1159
Epoch: 059, Loss: 0.1154
Epoch: 060, Loss: 0.1150
Epoch: 061, Loss: 0.1149
Epoch: 062, Loss: 0.1135
Epoch: 063, Loss: 0.1138
Epoch: 064, Loss: 0.1126
Epoch: 065, Loss: 0.1122
Epoch: 066, Loss: 0.1119
Epoch: 067, Loss: 0.1112
Epoch: 068, Loss: 0.1107
Epoch: 069, Loss: 0.1100
Epoch: 070, Loss: 0.1091
Epoch: 071, Loss: 0.1089
Epoch: 072, Loss: 0.1093
Epoch: 073, Loss: 0.1084
Epoch: 074, Loss: 0.1075
Epoch: 075, Loss: 0.1073


...

Epoch: 175, Val Mean Rank: 275.25, Val MRR: 0.3705
Epoch: 176, Loss: 0.0882
Epoch: 177, Loss: 0.0868
Epoch: 178, Loss: 0.0877
Epoch: 179, Loss: 0.0870
Epoch: 180, Loss: 0.0867
Epoch: 181, Loss: 0.0877
Epoch: 182, Loss: 0.0869
Epoch: 183, Loss: 0.0871
Epoch: 184, Loss: 0.0876
Epoch: 185, Loss: 0.0864
Epoch: 186, Loss: 0.0872
Epoch: 187, Loss: 0.0873
Epoch: 188, Loss: 0.0874
Epoch: 189, Loss: 0.0860
Epoch: 190, Loss: 0.0866
Epoch: 191, Loss: 0.0861
Epoch: 192, Loss: 0.0861
Epoch: 193, Loss: 0.0874
Epoch: 194, Loss: 0.0863
Epoch: 195, Loss: 0.0863
Epoch: 196, Loss: 0.0860
Epoch: 197, Loss: 0.0862
Epoch: 198, Loss: 0.0861
Epoch: 199, Loss: 0.0857
Epoch: 200, Loss: 0.0852


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 17535/17535 [04:37<00:00, 63.24it/s]


Epoch: 200, Val Mean Rank: 267.16, Val MRR: 0.3674


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20466/20466 [05:22<00:00, 63.47it/s]

Test Mean Rank: 284.64, Test MRR: 0.3673, 


#### RotatE


In [6]:
# training and testing rotate
model_type = 'rotate'
model_map = {
    'transe': TransE,
    'complex': ComplEx,
    'distmult': DistMult,
    'rotate': RotatE,
}

device = 'cuda' if torch.cuda.is_available() else 'cpu'
path = 'data'

train_data = FB15k_237(path, split='train')[0].to(device)
val_data = FB15k_237(path, split='val')[0].to(device)
test_data = FB15k_237(path, split='test')[0].to(device)

model_arg_map = {'rotate': {'margin': 9.0}}
model = model_map[model_type](
    num_nodes=train_data.num_nodes,
    num_relations=train_data.num_edge_types,
    hidden_channels=50,
    **model_arg_map.get(model_type, {}),
).to(device)

loader = model.loader(
    head_index=train_data.edge_index[0],
    rel_type=train_data.edge_type,
    tail_index=train_data.edge_index[1],
    batch_size=1000,
    shuffle=True,
)

optimizer_map = {
    'transe': optim.Adam(model.parameters(), lr=0.01),
    'complex': optim.Adagrad(model.parameters(), lr=0.001, weight_decay=1e-6),
    'distmult': optim.Adam(model.parameters(), lr=0.0001, weight_decay=1e-6),
    'rotate': optim.Adam(model.parameters(), lr=1e-3),
}
optimizer = optimizer_map[model_type]


def train():
    model.train()
    total_loss = total_examples = 0
    for head_index, rel_type, tail_index in loader:
        optimizer.zero_grad()
        loss = model.loss(head_index, rel_type, tail_index)
        loss.backward()
        optimizer.step()
        total_loss += float(loss) * head_index.numel()
        total_examples += head_index.numel()
    return total_loss / total_examples


@torch.no_grad()
def test(data):
    model.eval()
    return model.test(
        head_index=data.edge_index[0],
        rel_type=data.edge_type,
        tail_index=data.edge_index[1],
        batch_size=20000,
        k=10,
    )
# training and testing rotate 
for epoch in range(1, 201):
    loss = train()
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}')
    if epoch % 25 == 0:
        rank, mrr = test(val_data)
        print(f'Epoch: {epoch:03d}, Val Mean Rank: {rank:.2f}, '
              f'Val MRR: {mrr:.4f}')
    if epoch % 100 == 0:
        rank, mrr = test(test_data)
        print(f'Test Mean Rank: {rank:.2f}, Test MRR: {mrr:.4f}, ')
        mrr = round(mrr*100, 4)
        torch.save(model.state_dict(), './mods/rotate_mods/rotate_fbk_epch'+str(epoch)+'_mrr'+str(mrr)+'.pt')

Epoch: 001, Loss: 4.1393
Epoch: 002, Loss: 3.5514
Epoch: 003, Loss: 2.9819
Epoch: 004, Loss: 2.4961
Epoch: 005, Loss: 2.1159
Epoch: 006, Loss: 1.8190
Epoch: 007, Loss: 1.5829
Epoch: 008, Loss: 1.3959
Epoch: 009, Loss: 1.2488
Epoch: 010, Loss: 1.1283
Epoch: 011, Loss: 1.0320
Epoch: 012, Loss: 0.9508
Epoch: 013, Loss: 0.8837
Epoch: 014, Loss: 0.8278
Epoch: 015, Loss: 0.7805
Epoch: 016, Loss: 0.7413
Epoch: 017, Loss: 0.7072
Epoch: 018, Loss: 0.6783
Epoch: 019, Loss: 0.6542
Epoch: 020, Loss: 0.6335
Epoch: 021, Loss: 0.6156
Epoch: 022, Loss: 0.6005
Epoch: 023, Loss: 0.5875
Epoch: 024, Loss: 0.5745
Epoch: 025, Loss: 0.5641


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 17535/17535 [02:07<00:00, 137.21it/s]


Epoch: 025, Val Mean Rank: 3159.37, Val MRR: 0.0248
Epoch: 026, Loss: 0.5525
Epoch: 027, Loss: 0.5429
Epoch: 028, Loss: 0.5328
Epoch: 029, Loss: 0.5233
Epoch: 030, Loss: 0.5137
Epoch: 031, Loss: 0.5038
Epoch: 032, Loss: 0.4937
Epoch: 033, Loss: 0.4846
Epoch: 034, Loss: 0.4738
Epoch: 035, Loss: 0.4644
Epoch: 036, Loss: 0.4537
Epoch: 037, Loss: 0.4440
Epoch: 038, Loss: 0.4332
Epoch: 039, Loss: 0.4230
Epoch: 040, Loss: 0.4133
Epoch: 041, Loss: 0.4034
Epoch: 042, Loss: 0.3927
Epoch: 043, Loss: 0.3820
Epoch: 044, Loss: 0.3721
Epoch: 045, Loss: 0.3627
Epoch: 046, Loss: 0.3526
Epoch: 047, Loss: 0.3430
Epoch: 048, Loss: 0.3336
Epoch: 049, Loss: 0.3241
Epoch: 050, Loss: 0.3158


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 17535/17535 [02:08<00:00, 136.73it/s]


Epoch: 050, Val Mean Rank: 1156.91, Val MRR: 0.2341
Epoch: 051, Loss: 0.3060
Epoch: 052, Loss: 0.2973
Epoch: 053, Loss: 0.2887
Epoch: 054, Loss: 0.2808
Epoch: 055, Loss: 0.2728
Epoch: 056, Loss: 0.2655
Epoch: 057, Loss: 0.2580
Epoch: 058, Loss: 0.2506
Epoch: 059, Loss: 0.2437
Epoch: 060, Loss: 0.2373
Epoch: 061, Loss: 0.2310
Epoch: 062, Loss: 0.2248
Epoch: 063, Loss: 0.2190
Epoch: 064, Loss: 0.2137
Epoch: 065, Loss: 0.2079
Epoch: 066, Loss: 0.2032
Epoch: 067, Loss: 0.1987
Epoch: 068, Loss: 0.1930
Epoch: 069, Loss: 0.1890
Epoch: 070, Loss: 0.1846
Epoch: 071, Loss: 0.1808
Epoch: 072, Loss: 0.1773
Epoch: 073, Loss: 0.1737
Epoch: 074, Loss: 0.1697
Epoch: 075, Loss: 0.1667


...

Epoch: 175, Val Mean Rank: 171.40, Val MRR: 0.4343
Epoch: 176, Loss: 0.0949
Epoch: 177, Loss: 0.0950
Epoch: 178, Loss: 0.0944
Epoch: 179, Loss: 0.0936
Epoch: 180, Loss: 0.0938
Epoch: 181, Loss: 0.0938
Epoch: 182, Loss: 0.0936
Epoch: 183, Loss: 0.0938
Epoch: 184, Loss: 0.0935
Epoch: 185, Loss: 0.0934
Epoch: 186, Loss: 0.0937
Epoch: 187, Loss: 0.0935
Epoch: 188, Loss: 0.0931
Epoch: 189, Loss: 0.0928
Epoch: 190, Loss: 0.0932
Epoch: 191, Loss: 0.0930
Epoch: 192, Loss: 0.0924
Epoch: 193, Loss: 0.0933
Epoch: 194, Loss: 0.0924
Epoch: 195, Loss: 0.0926
Epoch: 196, Loss: 0.0926
Epoch: 197, Loss: 0.0927
Epoch: 198, Loss: 0.0919
Epoch: 199, Loss: 0.0920
Epoch: 200, Loss: 0.0920


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 17535/17535 [02:09<00:00, 135.72it/s]


Epoch: 200, Val Mean Rank: 163.86, Val MRR: 0.4347


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20466/20466 [02:28<00:00, 137.41it/s]

Test Mean Rank: 173.50, Test MRR: 0.4295, 


#### Complex 

In [7]:
# training and testing complex
model_type = 'complex'
model_map = {
    'transe': TransE,
    'complex': ComplEx,
    'distmult': DistMult,
    'rotate': RotatE,
}

device = 'cuda' if torch.cuda.is_available() else 'cpu'
path = 'data'

train_data = FB15k_237(path, split='train')[0].to(device)
val_data = FB15k_237(path, split='val')[0].to(device)
test_data = FB15k_237(path, split='test')[0].to(device)

model_arg_map = {'rotate': {'margin': 9.0}}
model = model_map[model_type](
    num_nodes=train_data.num_nodes,
    num_relations=train_data.num_edge_types,
    hidden_channels=50,
    **model_arg_map.get(model_type, {}),
).to(device)

loader = model.loader(
    head_index=train_data.edge_index[0],
    rel_type=train_data.edge_type,
    tail_index=train_data.edge_index[1],
    batch_size=1000,
    shuffle=True,
)

optimizer_map = {
    'transe': optim.Adam(model.parameters(), lr=0.01),
    'complex': optim.Adagrad(model.parameters(), lr=0.001, weight_decay=1e-6),
    'distmult': optim.Adam(model.parameters(), lr=0.0001, weight_decay=1e-6),
    'rotate': optim.Adam(model.parameters(), lr=1e-3),
}
optimizer = optimizer_map[model_type]


def train():
    model.train()
    total_loss = total_examples = 0
    for head_index, rel_type, tail_index in loader:
        optimizer.zero_grad()
        loss = model.loss(head_index, rel_type, tail_index)
        loss.backward()
        optimizer.step()
        total_loss += float(loss) * head_index.numel()
        total_examples += head_index.numel()
    return total_loss / total_examples


@torch.no_grad()
def test(data):
    model.eval()
    return model.test(
        head_index=data.edge_index[0],
        rel_type=data.edge_type,
        tail_index=data.edge_index[1],
        batch_size=20000,
        k=10,
    )
# training and testing complex 
for epoch in range(1, 201):
    loss = train()
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}')
    if epoch % 25 == 0:
        rank, mrr = test(val_data)
        print(f'Epoch: {epoch:03d}, Val Mean Rank: {rank:.2f}, '
              f'Val MRR: {mrr:.4f}')
    if epoch % 50 == 0:
        rank, mrr = test(test_data)
        print(f'Test Mean Rank: {rank:.2f}, Test MRR: {mrr:.4f}, ')
        mrr = round(mrr*100, 4)
        torch.save(model.state_dict(), './mods/complex_mods/complex_fbk_epch'+str(epoch)+'_mrr'+str(mrr)+'.pt')

Epoch: 001, Loss: 0.6931
Epoch: 002, Loss: 0.6931
Epoch: 003, Loss: 0.6931
Epoch: 004, Loss: 0.6931
Epoch: 005, Loss: 0.6931
Epoch: 006, Loss: 0.6931
Epoch: 007, Loss: 0.6931
Epoch: 008, Loss: 0.6930
Epoch: 009, Loss: 0.6930
Epoch: 010, Loss: 0.6928
Epoch: 011, Loss: 0.6927
Epoch: 012, Loss: 0.6925
Epoch: 013, Loss: 0.6923
Epoch: 014, Loss: 0.6920
Epoch: 015, Loss: 0.6916
Epoch: 016, Loss: 0.6912
Epoch: 017, Loss: 0.6907
Epoch: 018, Loss: 0.6901
Epoch: 019, Loss: 0.6895
Epoch: 020, Loss: 0.6888
Epoch: 021, Loss: 0.6881
Epoch: 022, Loss: 0.6873
Epoch: 023, Loss: 0.6865
Epoch: 024, Loss: 0.6855
Epoch: 025, Loss: 0.6846


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 17535/17535 [01:23<00:00, 209.86it/s]


Epoch: 025, Val Mean Rank: 694.40, Val MRR: 0.3080
Epoch: 026, Loss: 0.6836
Epoch: 027, Loss: 0.6825
Epoch: 028, Loss: 0.6814
Epoch: 029, Loss: 0.6803
Epoch: 030, Loss: 0.6791
Epoch: 031, Loss: 0.6779
Epoch: 032, Loss: 0.6767
Epoch: 033, Loss: 0.6754
Epoch: 034, Loss: 0.6741
Epoch: 035, Loss: 0.6727
Epoch: 036, Loss: 0.6714
Epoch: 037, Loss: 0.6700
Epoch: 038, Loss: 0.6686
Epoch: 039, Loss: 0.6672
Epoch: 040, Loss: 0.6657
Epoch: 041, Loss: 0.6643
Epoch: 042, Loss: 0.6628
Epoch: 043, Loss: 0.6613
Epoch: 044, Loss: 0.6598
Epoch: 045, Loss: 0.6582
Epoch: 046, Loss: 0.6567
Epoch: 047, Loss: 0.6553
Epoch: 048, Loss: 0.6538
Epoch: 049, Loss: 0.6522
Epoch: 050, Loss: 0.6506


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 17535/17535 [01:23<00:00, 210.51it/s]


Epoch: 050, Val Mean Rank: 673.07, Val MRR: 0.3155


...

Epoch: 175, Val Mean Rank: 643.29, Val MRR: 0.3372
Epoch: 176, Loss: 0.5317
Epoch: 177, Loss: 0.5320
Epoch: 178, Loss: 0.5309
Epoch: 179, Loss: 0.5308
Epoch: 180, Loss: 0.5303
Epoch: 181, Loss: 0.5296
Epoch: 182, Loss: 0.5295
Epoch: 183, Loss: 0.5292
Epoch: 184, Loss: 0.5287
Epoch: 185, Loss: 0.5278
Epoch: 186, Loss: 0.5282
Epoch: 187, Loss: 0.5274
Epoch: 188, Loss: 0.5271
Epoch: 189, Loss: 0.5270
Epoch: 190, Loss: 0.5265
Epoch: 191, Loss: 0.5259
Epoch: 192, Loss: 0.5256
Epoch: 193, Loss: 0.5249
Epoch: 194, Loss: 0.5249
Epoch: 195, Loss: 0.5244
Epoch: 196, Loss: 0.5233
Epoch: 197, Loss: 0.5238
Epoch: 198, Loss: 0.5228
Epoch: 199, Loss: 0.5222
Epoch: 200, Loss: 0.5229


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 17535/17535 [01:25<00:00, 204.52it/s]


Epoch: 200, Val Mean Rank: 634.85, Val MRR: 0.3414


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20466/20466 [01:38<00:00, 208.73it/s]

Test Mean Rank: 642.01, Test MRR: 0.3370, 
